In [ ]:
    import numpy as np
import pandas as pd

In [ ]:
import pandas as pd
import numpy as np
import nltk

In [3]:
amzon_df = pd.read_csv('C:/Users/HP/OneDrive/Documents/recommendation/amazon_product.csv')

In [4]:
amzon_df.head()

id                                              Title  \
0   1   Swissmar Capstore Select Storage Rack for 18-...   
1   2   Gemini200 Delta CV-880 Gold Crown Livery Airc...   
2   5   Superior Threads 10501-2172 Magnifico Cream P...   
3   6            Fashion Angels Color Rox Hair Chox Kit    
4   8   Union Creative Giant Killing Figure 05: Daisu...   

                                         Description  \
0  Swissmar's capstore select 18 storage unit kee...   
1  Welcome to the exciting world of GeminiJets! O...   
2  For quilting and embroidery, this product is m...   
3  Experiment with the haute trend of hair chalki...   
4  From Union Creative. Turn your display shelf i...   

                                            Category  
0   Home & Kitchen Kitchen & Dining Kitchen Utens...  
1   Toys & Games Hobbies Models & Model Kits Pre-...  
2   Arts, Crafts & Sewing Sewing Thread & Floss S...  
3   Beauty & Personal Care Hair Care Hair Colorin...  
4   Toys & Games › Action Figures & Statues › Sta...

In [5]:
amzon_df.drop('id',axis=1)

Title  \
0     Swissmar Capstore Select Storage Rack for 18-...   
1     Gemini200 Delta CV-880 Gold Crown Livery Airc...   
2     Superior Threads 10501-2172 Magnifico Cream P...   
3              Fashion Angels Color Rox Hair Chox Kit    
4     Union Creative Giant Killing Figure 05: Daisu...   
..                                                 ...   
663   Rosemery (Rosemary) - Box of Six 20 Stick Hex...   
664   InterDesign Linus Stacking Organizer Bin, Ext...   
665   Gourmet Rubber Stamps Diagonal Stripes Stenci...   
666   Spenco RX Arch Cushion Full Length Comfort Su...   
667                                           Car Kits   

                                           Description  \
0    Swissmar's capstore select 18 storage unit kee...   
1    Welcome to the exciting world of GeminiJets! O...   
2    For quilting and embroidery, this product is m...   
3    Experiment with the haute trend of hair chalki...   
4    From Union Creative. Turn your display shelf i...   
..                                                 ...   
663  Six tubes, each containing 20 sticks of incens...   
664  The InterDesign Linus Organizer Bins are stack...   
665  Gourmet Rubber Stamps-Stencil. This delicious ...   
666  Soft, durable arch support. consumers with gen...   
667                                       Kits for Car   

                                              Category  
0     Home & Kitchen Kitchen & Dining Kitchen Utens...  
1     Toys & Games Hobbies Models & Model Kits Pre-...  
2     Arts, Crafts & Sewing Sewing Thread & Floss S...  
3     Beauty & Personal Care Hair Care Hair Colorin...  
4     Toys & Games › Action Figures & Statues › Sta...  
..                                                 ...  
663   Home & Kitchen Home Décor Home Fragrance Ince...  
664   Home & Kitchen Kitchen & Dining Storage & Org...  
665    Toys & Games Arts & Crafts Printing & Stamping   
666   Health & Household › Health Care › Foot Healt...  
667                                    Car Accessories  

[668 rows x 3 columns]

In [6]:
amzon_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 668 entries, 0 to 667
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           668 non-null    int64 
 1   Title        668 non-null    object
 2   Description  668 non-null    object
 3   Category     668 non-null    object
dtypes: int64(1), object(3)
memory usage: 21.0+ KB


In [7]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

def tokenize_stem(text):
    tokens = nltk.word_tokenize(text.lower())
    stem = [stemmer.stem(w) for w in tokens]
    return " ".join(stem)

In [8]:
amzon_df['stemmed_tokens'] = amzon_df.apply(lambda row: tokenize_stem(row['Title'] + ' ' + row['Description']), axis=1)

In [9]:
amzon_df.head(2)

id                                              Title  \
0   1   Swissmar Capstore Select Storage Rack for 18-...   
1   2   Gemini200 Delta CV-880 Gold Crown Livery Airc...   

                                         Description  \
0  Swissmar's capstore select 18 storage unit kee...   
1  Welcome to the exciting world of GeminiJets! O...   

                                            Category  \
0   Home & Kitchen Kitchen & Dining Kitchen Utens...   
1   Toys & Games Hobbies Models & Model Kits Pre-...   

                                      stemmed_tokens  
0  swissmar capstor select storag rack for 18-pac...  
1  gemini200 delta cv-880 gold crown liveri aircr...

In [10]:
amzon_df['stemmed_tokens']

0      swissmar capstor select storag rack for 18-pac...
1      gemini200 delta cv-880 gold crown liveri aircr...
2      superior thread 10501-2172 magnifico cream puf...
3      fashion angel color rox hair chox kit experi w...
4      union creativ giant kill figur 05 : daisuk tsu...
                             ...                        
663    rosemeri ( rosemari ) - box of six 20 stick he...
664    interdesign linus stack organ bin , extra larg...
665    gourmet rubber stamp diagon stripe stencil , 6...
666    spenco rx arch cushion full length comfort sup...
667                                  car kit kit for car
Name: stemmed_tokens, Length: 668, dtype: object

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidvectorizer = TfidfVectorizer(tokenizer=tokenize_stem)

def cosine_sim(txt1,txt2):
    tfid_matrix = tfidvectorizer.fit_transform([txt1,txt2])
    return cosine_similarity(tfid_matrix)[0][1]

In [12]:
def search_product(query):
    stemmed_query = tokenize_stem(query)
    #calcualting cosine similarity between query and stemmed tokens columns
    amzon_df['similarity'] = amzon_df['stemmed_tokens'].apply(lambda x:cosine_sim(stemmed_query,x))
    res = amzon_df.sort_values(by=['similarity'],ascending=False).head(10)[['Title','Description','Category']]
    return res

In [13]:
search_product(' PURELL ES8 Professional HEALTHY SOAP Foam Refill, Fresh Scent Fragrance, 1200 mL Soap Refill for PURELL ES8 Touch-Free Dispenser (Pack of 2) - 7777-02 ')

C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Title  \
10    PURELL ES8 Professional HEALTHY SOAP Foam Ref...   
541         Remington SP290 for F4790 Shaver (2-Pack)    
176   Angel Soft Professional Series Premium 2-Ply ...   
160   Dixie 8.5"Medium-Weight Paper Plates by GP PR...   
206   Filofax 2018 Personal/Compact Academic Week t...   
220   Serta iComfort Premium Infant Sleeper Replace...   
492   Seventh Generation Baby Overnight Diapers, Fr...   
209   ALEX Toys Artist Studio Ultimate Easel Access...   
523   School Smart 2-Pocket Folders, Green, Pack of...   
390   Elmer's Foam Board, 20 x 28 Inches, 3/16 Inch...   

                                           Description  \
10   1200 ml refill for Purell ES8 touch-free soap ...   
541  Technical Features for Remington SP290-2 The R...   
176  Angel Soft Professional Series offers a qualit...   
160  The WiseSize product offering provides a packa...   
206  Over the years, the Filofax organizer has evol...   
220  Replacement cover for the Serta’s icomfort Pre...   
492  Everyone could use a little sleep. Seventh Gen...   
209  ALEX Toys Artist Studio Ultimate Easel Accesso...   
523  Compile and file with ease with School Smart's...   
390  Elmer's Foam Board makes it easier to create e...   

                                              Category  
10    Industrial & Scientific › Janitorial & Sanita...  
541   Beauty & Personal Care › Shave & Hair Removal...  
176   Health & Household Household Supplies Paper &...  
160   Health & Household › Household Supplies › Pap...  
206   Office Products › Office & School Supplies › ...  
220   Baby Products › Nursery › Bedding › Baby Bedd...  
492        Baby Products Diapering Disposable Diapers   
209                        Toys & Games Arts & Crafts   
523   Office Products Office & School Supplies Fili...  
390   Office Products Office & School Supplies Pres...

In [14]:
amzon_df['Title'][10]

' PURELL ES8 Professional HEALTHY SOAP Foam Refill, Fresh Scent Fragrance, 1200 mL Soap Refill for PURELL ES8 Touch-Free Dispenser (Pack of 2) - 7777-02 '